# Source Localization using Distributed Sensors (Focus on one Sensor)
 **(Final version will read directly from Pyserial and data will be collected at higher sampling rate)**

The first thing I need is a position matrix for all the sensors. The reason to start from a position matrix and not a distance matrix is because this algorithm can be adapted to a scenario where the sensors are not in a static position. I will record those positions in a dictionary using as relative position the position of sensor 1.

In [17]:
%%html
<img src="sensors.jpg" alt="Drawing" style="width: 500px;" align= "middle">

In [18]:
import numpy as np
import scipy
from matplotlib import pyplot
from scipy import signal
from scipy.stats import norm
from collections import deque
import random
import scipy.sparse.linalg as splinalg

In [19]:
def R_dictionary(sensor, Hash):
    Hash.update(sensor)

#Growing my Position dictionary:
Hash={}
k=0
for j in np.array([0, 15, 30, 45]):
    for i in np.array([0, 10, 20]):
        k=k+1
        R_dictionary({k: [j, i]}, Hash)
print(Hash)

{1: [0, 0], 2: [0, 10], 3: [0, 20], 4: [15, 0], 5: [15, 10], 6: [15, 20], 7: [30, 0], 8: [30, 10], 9: [30, 20], 10: [45, 0], 11: [45, 10], 12: [45, 20]}


Now, I want to create a relative distance matrix by computing the Euclidean distance between each sensor. 

In [20]:
Rel_Pos= np.zeros((len(Hash), len(Hash)))
for i in  np.arange(len(Hash)):
    for j in np.arange(len(Hash)):
        Rel_Pos[i][j]= ((Hash[i+1][0] - Hash[j+1][0])**2 + (Hash[i+1][1] - Hash[j+1][1])**2)**(1./2.)
#pyplot.spy(Rel_Pos)
#pyplot.show()

TO DO:
* ~~Import the data and parse it~~
* ~~Make a routine that decomposes the signal at level 1: s= a1+d1~~
* ~~Use d1 to determine if there is an event if there is no event, don't do anything. If there is an event, fire the algorithm and record the time in which the event happened (t!)~~
* ~~With the data identify the ones that got highest frequency at t!~~
* ~~Assign probabilities to each sensor~~ ~~Breed~~
* ~~Do the triangulization method~~
* Do the things suggested by Jed in the gitter
    * ~~What does order of accuracy mean in this context?~~
    * How would equations be if the medium is not isotropic so time delay is not just linear distance?
    * Eikonal Equation

A common way to localize sources is using a method called the Triangularization Method. This means that one sensor communicates to two other sensors in order to localize where the source of the wave came from. The main Equation to be solved here takes the following form:

\begin{equation}
E(x_0, y_0)= 2(x_0^2+y_0^2-x_0x_2-x_0x_3-y_0y_2-y_0y_3) + x_2^2+x_3^2+y_2^2+y_3^2 - (r_{21}^2+r_{31}^2)(x_0^2+y_0^2-2(x_0x_1+y_0y_1)+x_1^2+y_1^2))^2
\end{equation}

In [25]:
def Event(sensor, right):
    # Module for Anomaly Detection
    # For CSCI 5363, I will use a Discrete Wavelet Transform for this module using the Fast Wavelet Transform Algorithm
    # I only care about the level 1 decomposition's second coefficient (d1) in  s= a1+d1 to identify whether there is an event or not
    # For CNN purposes, this module will be equivalent to the convolutional layer in a neural network
    widths = np.arange(1, 31)
    sensor= np.array(sensor)
    cwtmatr = signal.cwt(sensor.astype(int), signal.ricker, widths)
    yesno=0
    left=right
    right = np.linalg.norm(cwtmatr[:, -1], np.inf)
    
    if right-left > 300:
        yesno = 1 
        sensdata= sensor[-1]   
    else:
        yesno = 0
        sensdata = np.zeros_like(right)
        
    return yesno, right, sensdata

def F(x1, x2, x3, y1, y2, y3, r_21, r_31, u):
    x0= u[0]
    y0= u[1]
    return np.array([2*(x0**2 + y0**2 - x0*x2 -x0*x3 -y0*y2 -y0*y3)+ x2**2+ x3**2, 
                 y2**2 + y3**2 -(r_21**2+r_31**2)*(x0**2+y0**2-2*(x0*x1+y0*y1)
                                                + x1**2 + y1**2)])

def J(x1, x2, x3, y1, y2, y3,  r_21, r_31, u):
    x0= u[0]
    y0= u[1]
    return np.array([[2*(2*x0-x2-x3), 2*(2*y0-y2-y3)], 
                    [-(r_21**2+r_31**2)*(2*x0-2*x1), -(r_21**2+r_31**2)*(2*y0-2*y1)]])

#def F1(x1, x2, x3, y1, y2, y3, r_21, r_31,r_23, u):
#    x0= u[0]
#    y0= u[1]
#    d1= ((x1-x0)**2+(y1-y0)**2)**(1/2)
#    d2= ((x2-x0)**2+(y2-y0)**2)**(1/2)
#    d3= ((x3-x0)**2+(y3-y0)**2)**(1/2)
#    return np.array([(r_23*(d1-d2) - r_21*(d2-d2))**2 + (r_31*(d2-d3) - r_23*(d3-d1))**2+ (r_21*(d3-d1) - r_31*(d1-d2))**2])

#def J1(x1, x2, x3, y1, y2, y3,  r_21, r_31, u):
#    x0= u[0]
#   y0= u[1]
#    d1dx= ((x1-x0)**2+(y1-y0)**2)**(1/2)
#    d2= ((x2-x0)**2+(y2-y0)**2)**(1/2)
#    d3= ((x3-x0)**2+(y3-y0)**2)**(1/2)
#    return np.array([])

    
def SourceLoc(s1,s2,s3,u,x1,x2,x3,y1,y2,y3,rtol,maxit,epsilon,verbose):
    #print(s1, s2, s3)
    # This is my version of fsolve_newton() by Jed Brown
    # This is the actual routine that will do the
    # source localization given three sensors and
    # time of arrival
    u0= u.copy()
    
    #I will be using the cross correlation to get the time delay of the signals
    r_21 = signal.correlate(s1,s2, mode='same')/(360*400)
    r_31 = signal.correlate(s1,s3,mode='same')/(360*400)
    
   # print(r_21, r_31)
    
    Fu= F(x1, x2, x3, y1, y2, y3, r_21, r_31, u)
    JJ= J(x1, x2, x3, y1, y2, y3, r_21, r_31, u)
    
    norm0= np.linalg.norm(Fu)
    enorm_last= np.linalg.norm(u - np.array([1,1],dtype= np.float64))
    for i in range(maxit):
        
        def Ju_fd(v): # Preconditioning the Jacobian
            return (F(x1, x2, x3, y1, y2, y3, r_21, r_31, u + epsilon*v) - 
                F(x1, x2, x3, y1, y2, y3, r_21, r_31, u)) / epsilon
    
        Ju = splinalg.LinearOperator((len(Fu),len(u)), matvec=Ju_fd)
    
        du, info = splinalg.gmres(Ju, Fu)
    
        if info != 0:
            raise RuntimeError('GMRES failed to converge: {:d}'.format(info))
        
        u -= du

        Fu= F(x1, x2, x3, y1, y2, y3, r_21, r_31, u)
        norm= np.linalg.norm(Fu)
        
        if verbose:
            enorm= np.linalg.norm(u - np.array([1,1]))
            print('Newton {:d} anorm {:6.2e} rnorm {:6.2e} eratio {:6.2f}'.
                  format(i+1, norm, norm/norm0, enorm/enorm_last**2))
            enorm_last= enorm
        if norm < rtol*norm0:
            break
        if np.isnan(norm):
            raise RuntimeError('Newton Raphson failed to converge: {:d}'.format(info)) 
    return u,i

def Breeding(all_sensors, n):
    #This function will be called by each sensor every time there is an event in order
    #to be paired with two of the fittest sensors to make the guess of the source localization

    sensordata1, sensordata2, sensordata3, sensordata4, sensordata5, sensordata6= all_sensors
   
    listing= np.arange(len(all_sensors))
    listing= np.delete(listing, 0, n)
    
    fitness= np.array(all_sensors)
    fitness= np.delete(fitness, 0, n)
    fitness= fitness.astype(np.float) #I will put that fitness into a probability density function
    while np.count_nonzero(fitness)==1:
        fitness[np.random.randint(1,len(fitness))]= 10.0
    p=fitness/sum(fitness)
    
    pair= np.random.choice(listing,2, replace=False, p = p) 
    
    return pair 
    

In [ ]:
f= 'Data1st.txt' #just importing data from touch in a single point with different forces
# This dataset has the peculiarity that I only touched the same spot ten times, therefore, my estimations should
# be close to those coordinates.
window_size= 1000
sampling_frequency= 400 #Hz
Window1= deque([])
Window2= deque([])
Window3= deque([])
Window4= deque([])
Window5= deque([])
Window6= deque([])
j=0
# Each line is a time stamp. 
# The sampling rate at which this data was taken is 400 Hz and the goal is 1kHz
right1=0
right2=0
right3=0
right4=0
right5=0
right6=0

#im = pyplot.imread('sensors.png')
#implot = pyplot.imshow(im)
pyplot.figure()
with open(f) as file: #this loop will be replaced by some pyserial readline instruction if a life demo were to be done
    for line in file:
        data, timestamp = line.split("-")
        Sensors= data.split(",")
        
        #Because this is a simulation of the "big picture", I am going to use 6 deques:
        Sensors1=Sensors[0]
        Sensors2=Sensors[1]
        Sensors3=Sensors[2]
        Sensors4=Sensors[3]
        Sensors5=Sensors[4]
        Sensors6=Sensors[5]
        
        if j<=window_size:    
            Window1.append(Sensors1)
            Window2.append(Sensors2)
            Window3.append(Sensors3)
            Window4.append(Sensors4)
            Window5.append(Sensors5)
            Window6.append(Sensors6)
            j= j+1
        else:
            Window1.append(Sensors1)
            Window2.append(Sensors2)
            Window3.append(Sensors3)
            Window4.append(Sensors4)
            Window5.append(Sensors5)
            Window6.append(Sensors6)
            Window1.popleft()
            Window2.popleft()
            Window3.popleft()
            Window4.popleft()
            Window5.popleft()
            Window6.popleft()  
            
# Now that the windows are ready, I will send them to the Event function and if most of the sensors agree
# that there is an event, then the breeding pairing will actuate

        if j>= window_size:
            yesno1, right1, sensdata1= Event(Window1, right1)
            sensdata2= Window2[-1]
            sensdata3= Window3[-1]
            sensdata4= Window4[-1]
            sensdata5= Window5[-1]
            sensdata6= Window6[-1]
  
            if yesno1==1:
                listsens= (sensdata1, sensdata2, sensdata3, sensdata4, sensdata5, sensdata6)
                pair1= Breeding(listsens, 0) #sensor1 will start at 0
                x1= Hash[1][0]
                y1= Hash[1][1]
                x2= Hash[pair1[0]+1][0]
                y2= Hash[pair1[0]+1][1]
                x3= Hash[pair1[1]+1][0]
                y3= Hash[pair1[1]+1][1]
                s1= sensdata1
                s2= listsens[pair1[0]]
                s3= listsens[pair1[1]]
                
                #print(x1,y1,x2,y2,x3,y3)

                u,i= SourceLoc(float(s1),float(s2),float(s3),[1,1],x1,x2,x3,y1,y2,y3,rtol=1e-10,maxit=500,
                               epsilon=1e-8,verbose= False)
                
                # put a red dot, size 40, at 2 locations:
                pyplot.scatter(u[0], u[1], c='k', s=40)
                 
                pyplot.xlim(xmin=-5)
                pyplot.xlim(xmax= 19)
                pyplot.ylim(ymin= -5)
                pyplot.ylim(ymax= 7)
                print(u)  
                
    pyplot.scatter(2.5, 2, c='r', s=40)          
pyplot.show()
        #work from here to localize the source
        #yesno= Event(Window, sensor_index)

[ 12.17142158   2.87862135]


## What does order of accuracy mean in this context?

The means to solve this problem was the Newton Raphson algorithm. Newton Raphson's essence is to solve a complicated equation by optimizing an expression. However, it is unknown whether the optimization of this expression is actually the solution to the problem or not. 

** Equations in the case of an anisotropic medium **


In the case of an anisotropic medium, the time delay would not be linear. This means that the shear waves would split. 